In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
from pyspark.sql.functions import greatest, lower, upper, trim,least
from pyspark.sql.functions import collect_set, col, lpad, lit, collect_list, desc, asc, mean as sql_mean, sum as sql_sum, datediff, count as sql_count
#from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField, IntegerType
from pyspark.sql.functions import array, regexp_extract, datediff, to_date, from_unixtime, unix_timestamp, desc, when, col, lit, udf, size, \
    array, isnan, upper, coalesce, length, lower, concat, create_map, sum as sql_sum, greatest, max as sql_max, sort_array, min as sql_min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
import re
from pyspark.sql.types import ArrayType, FloatType, StringType
from pyspark.sql.functions import datediff
import numpy as np
from pyspark.sql.types import FloatType
from pyspark.sql.functions import lag, mean as sql_avg

In [3]:
class UDFconversion:

    @staticmethod
    def convert(s_orig):
        import re
        if not s_orig: return ""
        s = s_orig
        s = re.sub("instalaci.n", "instalacion", s)
        s = re.sub("m.vil", "movil", s)
        s = re.sub("desconexi.n", "desconexion", s)
        s = re.sub("disminuci.n", "disminucion", s)
        s = re.sub("reconexi.n", "reconexion", s)
        s = re.sub("suspensi.n", "suspension", s)
        s = re.sub("reconexi.n", "reconexion", s)
        s = re.sub("aplicaci.n", "aplicacion", s)
        s = re.sub("numeraci.n", "numeracion", s)
        s = re.sub("tel.fono", "telefono", s)
        s = re.sub("b.sico", "basico", s)
        s = re.sub("telefon.a", "telefonia", s)
        s = re.sub("migraci.n", "migracion", s)
        s = re.sub("facturaci.n", "facturacion", s)
        s = re.sub("correcci.n", "correccion", s)
        s = re.sub("aver.a", "averia", s)
        s = re.sub("activaci.n", "activacion", s)
        s = re.sub("garant.a", "garantia", s)
        s = re.sub("modificaci.n", "modificacion", s)
        s = re.sub("n.mero", "numero", s)
        s = re.sub("fidelizaci.n", "fidelizacion", s)
        s = re.sub("t.cnica", "tecnica", s)
        s = re.sub("gesti.n", "gestion", s)
        s = re.sub("reclamaci.n", "reclamacion", s)
        s = re.sub("planificaci.n", "planificacion", s)
        s = re.sub("configuraci.n", "configuracion", s)
        s = re.sub("reprovisi.n", "reprovision", s)
        # s = re.sub("", "", s)
        # s = re.sub("", "", s)
        return s


In [4]:
def get_sla_master_df(spark):


    convert_udf = udf(UDFconversion.convert, StringType())

    df_sla = spark.read.option("header", True).option("delimiter", ";").option("encoding", "utf-8").csv \
        ("/data/udf/vf_es/churn/SLA/TorreControl_SLA.csv").where \
        (col("TITULO").isNotNull( ) &col("CLASIFICACION").isNotNull( ) &col("TIPO ORDEN").isNotNull())
    df_sla = df_sla.withColumn('TITULO', trim(lower(col('TITULO'))))
    df_sla = df_sla.withColumn('CLASIFICACION', trim(lower(col('CLASIFICACION'))))
    df_sla = df_sla.withColumn('TIPO ORDEN', trim(upper(col('TIPO ORDEN'))))
    df_sla = df_sla.withColumn("TITULO", convert_udf(col("TITULO")))
    df_sla = df_sla.withColumn("SLA Negocio presencial", when(col("SLA Negocio presencial").isNull() ,-1).otherwise(col("SLA Negocio presencial")))
    df_sla = df_sla.withColumn("SLA Negocio no presencial", when(col("SLA Negocio no presencial").isNull() ,-1).otherwise
        (col("SLA Negocio no presencial")))

    df_sla = df_sla.withColumn("SLA_master",  when( col("SLA Negocio presencial").isNotNull() & col("SLA Negocio no presencial").isNotNull(), greatest(col("SLA Negocio presencial"), col("SLA Negocio no presencial")))
                               .when( col("SLA Negocio presencial").isNotNull(), col("SLA Negocio presencial"))
                               .when( col("SLA Negocio no presencial").isNotNull(), col("SLA Negocio no presencial")).otherwise(-1))
    return df_sla



In [5]:
def get_orders_df(spark, closing_day, exclude_clasif_list=None):

    convert_udf = udf(UDFconversion.convert, StringType())

    from pykhaos.utils.date_functions import move_date_n_days
    starting_day = move_date_n_days(closing_day, n=-365)

    data_order_ori = (spark.read.load("/data/raw/vf_es/customerprofilecar/ORDERCRMOW/1.1/parquet/")
                      .where(col('fecha_entrada') <= closing_day)
                      .where(col('fecha_entrada') >= starting_day)
                      )

    w_orderclass = Window().partitionBy("OBJID").orderBy(desc("year"), desc("month"), desc("day"))

    data_orderclass_ori = (spark.read.load("/data/raw/vf_es/customerprofilecar/ORDERCLASOW/1.0/parquet/")
                           .where(
        concat(col('year'), lpad(col('month'), 2, '0'), lpad(col('day'), 2, '0')) <= closing_day)
                           .withColumn("rowNum", row_number().over(w_orderclass))
                           .where(col('rowNum') == 1)
                           .drop(*['year', 'month', 'day'])
                           )

    w_order = Window().partitionBy("orden_id").orderBy(desc("year"), desc("month"), desc("day"))

    data_orders_joined = (
        data_order_ori.join(data_orderclass_ori, on=col('clase_orden') == col('objid'), how='leftouter')
            .withColumn("rowNumorder", row_number().over(w_order))
            .where(col('rowNumorder') == 1)
    )

    data_orders_joined = data_orders_joined.cache()
    print("before join df_customer={}".format(data_orders_joined.count()))


    #######
    # Join with car to obtain NIF
    df_customer = spark.read.load(
        "/data/udf/vf_es/amdocs_ids/customer/year={}/month={}/day={}".format(int(closing_day[:4]),
                                                                             int(closing_day[4:6]),
                                                                             int(closing_day[6:])))
    # keep only the num_cliente in customer table
    data_orders_joined = data_orders_joined.join(df_customer.select("num_cliente", "NIF_CLIENTE"), on=["num_cliente"], how="inner")

    data_orders_joined = data_orders_joined.cache()
    print("after data_orders_joined={}".format(data_orders_joined.count()))

    data_orders_joined = data_orders_joined.withColumn('X_TIPO_ORDEN', trim(upper(col('X_TIPO_ORDEN'))))
    data_orders_joined = data_orders_joined.withColumn('x_clasificacion', trim(lower(col('x_clasificacion'))))
    data_orders_joined = data_orders_joined.withColumn('X_Description', trim(lower(col('X_Description'))))
    data_orders_joined = data_orders_joined.withColumn("X_Description", convert_udf(col("X_Description")))
    data_orders_joined = (data_orders_joined.withColumn("days_since_start", when(col("FECHA_ENTRADA").isNotNull(),
                                                                                 datediff(from_unixtime(
                                                                                     unix_timestamp(lit(closing_day),
                                                                                                    "yyyyMMdd")), col(
                                                                                     "FECHA_ENTRADA"))).otherwise(-1)))

    data_orders_joined = (data_orders_joined.withColumn("days_since_completed", when(
        col("FECHA_WO_COMPLETA").isNotNull() & (col("ESTADO_FIN_WO") == "C"),  # be sure is completed
        datediff(from_unixtime(unix_timestamp(lit(closing_day),
                                              "yyyyMMdd")), col("FECHA_WO_COMPLETA"))).otherwise(None))
                          # set to None, orders completed after closing_day
                          .withColumn("days_since_completed", when(col("days_since_completed") < 0, None).otherwise(
        col("days_since_completed")))
                          )

    # -----[----------------]-------
    #     a year        closing_day
    data_orders_joined = (data_orders_joined.where((col("days_since_start") <= 365) & (col("days_since_start") >= 0)))

    data_orders_joined = (data_orders_joined
                          .withColumn("duration",
                                      when((col("days_since_start") != -1) & (col("days_since_completed").isNotNull()),
                                           (col("days_since_start") - col("days_since_completed"))).otherwise(None))
                          )

    # FILTERING ENTRIES
    # remove orders with category "desconexion"
    data_orders_joined = data_orders_joined.where(~col("x_clasificacion").rlike("(?i)desconexion"))
    if exclude_clasif_list:
        for clasif in exclude_clasif_list:
            print(clasif)
            data_orders_joined = data_orders_joined.where(~col("x_clasificacion").rlike("(?i){}".format(clasif.lower())))
            print("Excluded x_clasification {}".format(clasif.lower()))

    # within the category "disminucion", remove those that contain "portabilidad" y "baja" inside the X_Description field
    data_orders_joined = data_orders_joined.where(~((col("x_clasificacion").rlike("(?i)disminucion")) &
                                                    (col("X_Description").rlike("(?i)portabilidad|baja|desconexion"))))

    data_orders_joined = data_orders_joined.withColumn("traslado", when(col("X_Description").rlike("(?i)traslado"), 1).otherwise(0))

    return data_orders_joined



In [6]:
def do_merge(spark, closing_day, exclude_clasif_list, days_range):
    data_orders_joined = get_orders_df(spark, closing_day, exclude_clasif_list)
    df_sla = get_sla_master_df(spark)

    df_order_master = data_orders_joined.join(df_sla.select("CLASIFICACION", 'TIPO ORDEN', "TITULO", "SLA_master"),
                                              on=(df_sla['TIPO ORDEN'] == data_orders_joined['X_TIPO_ORDEN']) &
                                                 (df_sla["CLASIFICACION"] == data_orders_joined["x_clasificacion"]) &
                                                 (df_sla["TITULO"] == data_orders_joined['X_Description']), how="left")

    # keep_gt_factor_udf = udf(lambda milista, factor: list([milista[ii] for ii in range(len(milista)) if milista[ii] > factor]), ArrayType(StringType()))
    #days_range = [30, 60, 90, 120, 180, 240, 365]
    # remove_undefined_udf = udf(lambda milista: list([milista[ii] for ii in range(len(milista)) if milista[ii] != -1]), ArrayType(FloatType()))
    # avg_days_bw_orders_udf = udf(lambda milista: float(np.mean([milista[ii + 1] - milista[ii] for ii in range(len(milista) - 1)])), FloatType())

    # mean_udf = udf(lambda milista: float(np.mean(milista)), FloatType())

    df_order = df_order_master.cache()

    for dd in days_range:
        df_order = df_order.withColumn("duration_last{}".format(dd),
                                       when((col("days_since_completed") <= dd) & (col("days_since_completed") != -1),
                                            col("duration")).otherwise(None))
        print("CSANC109", "duration_last{}".format(dd))
        df_order = df_order.withColumn("SLA_factor_last{}".format(dd),
                                       when(col("duration_last{}".format(dd)).isNotNull() & col(
                                           "SLA_master").isNotNull(),
                                            col("duration_last{}".format(dd)) / col("SLA_master")).otherwise(None))
        # to ignore an order, set the column "days_since..." to -1. following line, set "diff" value of days_since_started=-1 to Null
        # diff is the difference (in days) between an order and the next one
        df_order = df_order.withColumn("diff",
                                       when(col("days_since_start") != -1, lag(col("days_since_start"), -1).over(
                                           Window.partitionBy("NIF_CLIENTE").orderBy(asc("days_since_start"))) - col(
                                           "days_since_start")).otherwise(None))

        df_order = df_order.withColumn("days_since_start_traslados",
                                       when(col("traslado") == 1, col("days_since_start")).otherwise(None))
        df_order = df_order.withColumn("diff_traslados",
                                       when(((col("traslado") == 1) & (col("days_since_start_traslados") != -1)),
                                            lag(col("days_since_start_traslados"), -1).over(
                                                Window.partitionBy("NIF_CLIENTE").orderBy(
                                                    asc("days_since_start_traslados"))) - col(
                                                "days_since_start_traslados")).otherwise(None))

    return df_order

In [7]:
def get_orders_module(spark, closing_day, exclude_clasif_list=None, days_range=None):

    print("get_orders_module")

    print(days_range)
    if not days_range:
        days_range = [30, 60, 90, 120, 180, 240, 365]

    print("Using days_range {}".format(",".join(map(str, days_range))))

    df_order = do_merge(spark, closing_day, exclude_clasif_list, days_range)

    factor_list = [1, 2, 3]


    df_agg = (df_order.groupby("NIF_CLIENTE").agg(*([sql_count(when(((col("days_since_start") < dd) & (col("days_since_start") != -1)), col("days_since_start")).otherwise(None)).alias("nb_started_orders_last{}".format(dd)) for dd in days_range] +
                                                    [sql_count(when(col("duration_last{}".format(dd)).isNotNull(), col("duration_last{}".format(dd))).otherwise(None)).alias("nb_completed_orders_last{}".format(dd)) for dd in days_range] +
                                                    [sql_count(when(col("SLA_factor_last{}".format(dd)) > ff, col("SLA_factor_last{}".format(dd))).otherwise(None)).alias("nb_completed_orders_last{}_{}SLA".format(dd, ff)) for dd in days_range for ff in factor_list] +
                                                    [sql_avg(col("diff")).alias("avg_days_bw_open_orders")] +
                                                    # ignore null factors (when SLA_master is null)
                                                    [sql_avg(col("SLA_factor_last{}".format(dd))).alias("mean_sla_factor_last{}".format(dd)) for dd in days_range] +
                                                    # traslados
                                                    [sql_count(when((  (col("days_since_start_traslados").isNotNull()) & (col("days_since_start_traslados") < dd) ), col("days_since_start_traslados")).otherwise(None)).alias("nb_started_orders_traslados_last{}".format(dd)) for dd in days_range] +
                                                    [sql_count(when( ((col("traslado") == 1) & col("duration_last{}".format(dd)).isNotNull()), col("duration_last{}".format(dd))).otherwise(None)).alias("nb_completed_orders_traslados_last{}".format(dd)) for dd in days_range] +
                                                    [sql_count(when( ((col("traslado") == 1) & (col("SLA_factor_last{}".format(dd)) > ff)), col("SLA_factor_last{}".format(dd))).otherwise(None)).alias("nb_completed_orders_traslados_last{}_{}SLA".format(dd, ff)) for dd in days_range for ff in factor_list] +
                                                    [sql_avg(col("diff_traslados")).alias("avg_days_bw_open_orders_traslado")] +
                                                    # ignore null factors (when SLA_master is null)
                                                    [sql_avg(when( col("traslado") == 1,  col("SLA_factor_last{}".format(dd))).otherwise(None)).alias("mean_sla_factor_traslados_last{}".format(dd)) for dd in days_range]

                                                  )))
    return df_agg


In [8]:
def set_paths():
    '''
    Deployment should be something like "dirs/dir1/use-cases"
    This function adds to the path "dirs/dir1/use-cases" and "dirs/dir1/"
    :return:
    '''
    import imp
    from os.path import dirname
    import os
    import sys

    USE_CASES = "/var/SP/data/home/bgmerin1/src/Repositorios/use-cases"#dirname(os.path.abspath(imp.find_module('churn')[1]))

    if USE_CASES not in sys.path:
        sys.path.append(USE_CASES)
        print("Added '{}' to path".format(USE_CASES))

    # if deployment is correct, this path should be the one that contains "use-cases", "pykhaos", ...
    # FIXME another way of doing it more general?
    DEVEL_SRC = os.path.dirname(USE_CASES)  # dir before use-cases dir
    if DEVEL_SRC not in sys.path:
        sys.path.append(DEVEL_SRC)
        print("Added '{}' to path".format(DEVEL_SRC))
set_paths()

Added '/var/SP/data/home/bgmerin1/src/Repositorios/use-cases' to path
Added '/var/SP/data/home/bgmerin1/src/Repositorios' to path


In [9]:
import sys, os, re

pathname = os.path.dirname(sys.argv[0])  # abs path to file (not included)
print("pathname", pathname)

if pathname.startswith("/var/SP/data/bdpmdses/deliveries_churn/"):
    import re
    root_dir = re.match("^(.*)use-cases(.*)", pathname).group(1)
else:
    root_dir = re.match("(.*)use-cases/churn(.*)", pathname).group(1)
print("Detected '{}' as root dir".format(root_dir))

if root_dir not in sys.path:
    sys.path.append(root_dir)
    print("Added '{}' to path".format(root_dir))

mypath = os.path.join(root_dir, "use-cases")
if mypath not in sys.path:
    sys.path.append(mypath)
    print("Added '{}' to path".format(mypath))

('pathname', '/opt/cloudera/parcels/Anaconda-2.5.0/lib/python2.7/site-packages')


AttributeError: 'NoneType' object has no attribute 'group'

In [10]:
closing_day='20190521'
exclude_clasif=None

In [11]:
from churn.utils.general_functions import init_spark

spark = init_spark("sla_data_loader_{}".format(closing_day))

Ended spark session: 39.789757967 secs | default parallelism=4


In [12]:
if "," in closing_day:
    closing_day_list = closing_day.split(",")
else:
    closing_day_list = [closing_day]

if not exclude_clasif:
    exclude_clasif_list = None
elif exclude_clasif and "," in exclude_clasif:
    exclude_clasif_list = exclude_clasif.split(",")
else:
    exclude_clasif_list = [exclude_clasif]
    
print(closing_day_list)
print(exclude_clasif_list)

['20190521']
None


In [13]:
days_range = [30, 60, 90, 120, 180, 240, 365]

In [14]:
print("Using days_range {}".format(",".join(map(str, days_range))))

df_orderM = do_merge(spark, closing_day, exclude_clasif_list, days_range)

Using days_range 30,60,90,120,180,240,365
before join df_customer=20760101
after data_orders_joined=16180713
('CSANC109', 'duration_last30')
('CSANC109', 'duration_last60')
('CSANC109', 'duration_last90')
('CSANC109', 'duration_last120')
('CSANC109', 'duration_last180')
('CSANC109', 'duration_last240')
('CSANC109', 'duration_last365')


In [ ]:
df_orderM.columns

In [ ]:
(df_orderM.select(['days_since_start','days_since_completed','duration','duration_last30','duration_last60','duration_last120'])

.filter(col('duration')>30)
.show())


In [ ]:
df_orderM.select('days_since_completed').distinct().sort('days_since_completed').show()



In [15]:
df_orderM = df_orderM.withColumn("x_clasificacion", trim(col("x_clasificacion")))
df_orderM = df_orderM.withColumn("x_clasificacion", when(col("x_clasificacion").rlike("ord.*administrativas"),
                                                          "ord_admin").otherwise(col("x_clasificacion")))
df_orderM = df_orderM.withColumn("x_clasificacion", when(col("x_clasificacion").rlike("ord.*equipo"), "ord_equipo").otherwise(col("x_clasificacion")))
df_orderM = df_orderM.withColumn("x_clasificacion", when(col("x_clasificacion").rlike("porta.*hz"), "porta_hz").otherwise(col("x_clasificacion")))
df_orderM = df_orderM.withColumn("x_clasificacion", when(col("x_clasificacion").rlike("ordenes.*especiales"), "ord_esp").otherwise(col("x_clasificacion")))


In [ ]:
df_orderM.select('x_clasificacion').distinct().show()

In [16]:
from pyspark.sql.functions import count

df_orderMAttrib=df_orderM

df_orderMAttrib=df_orderMAttrib.withColumn('running_days',when(col('days_since_completed').isNotNull(),col('duration')).otherwise(col('days_since_start')))

df_orderMAttrib = (df_orderMAttrib.withColumn("SLA_factor_running",
                        when(col("running_days").isNotNull() & col("SLA_master").isNotNull(),
                                col("running_days") / col("SLA_master")).otherwise(None)))

columnas=['running_days','SLA_factor_running']

for dd in days_range:    
    
    df_orderMAttrib = df_orderMAttrib.withColumn("Flag_open_order{}".format(dd),
                        when((col('days_since_completed').isNull()) & (col('running_days') < dd),1).otherwise(0))
    
    df_orderMAttrib = df_orderMAttrib.withColumn("Flag_open_order_insideSLA{}".format(dd),
                        when((col("Flag_open_order{}".format(dd))==1) & (col('running_days') < (col('SLA_master'))),1).otherwise(0))
    
    df_orderMAttrib = df_orderMAttrib.withColumn("Flag_open_order_outsideSLA{}".format(dd),
                        when((col("Flag_open_order{}".format(dd))==1) & (col('running_days') > (col('SLA_master'))),1).otherwise(0))
    
    
    columnas.extend(['Flag_open_order{}'.format(dd)])
    
    #df_orderM.filter(col("Flag{}".format(dd))==1).groupBy('x_clasificacion').agg(count('*')).show()
columnas.extend(['NUM_CLIENTE','NIF_CLIENTE'])



In [17]:
days_range = [30, 60, 90, 120, 180, 240, 365]

for day in days_range:

    print 'Day:',day
    print 'N Ordenes abiertas',df_orderMAttrib.filter(col('Flag_open_order'+str(day))==1).count()
    print 'N clientes',df_orderMAttrib.filter(col('Flag_open_order'+str(day))==1).select('NIF_CLIENTE').distinct().count()

    print 'N Ordenes cerradas',df_orderMAttrib.filter(col('Flag_open_order'+str(day))==0).count()
    print 'N clientes',df_orderMAttrib.filter(col('Flag_open_order'+str(day))==0).select('NIF_CLIENTE').distinct().count()

    print '**********'

Day: 30
N Ordenes abiertas 131537
N clientes 92265
N Ordenes cerradas 11505691
N clientes 2535783
**********
Day: 60
N Ordenes abiertas 200852
N clientes 142499
N Ordenes cerradas 11436376
N clientes 2530809
**********
Day: 90
N Ordenes abiertas 235212
N clientes 168547
N Ordenes cerradas 11402016
N clientes 2528200
**********
Day: 120
N Ordenes abiertas 253304
N clientes 181642
N Ordenes cerradas 11383924
N clientes 2527013
**********
Day: 180
N Ordenes abiertas 265395
N clientes 190560
N Ordenes cerradas 11371833
N clientes 2525988
**********
Day: 240
N Ordenes abiertas 265395
N clientes 190560
N Ordenes cerradas 11371833
N clientes 2525988
**********
Day: 365
N Ordenes abiertas 265395
N clientes 190560
N Ordenes cerradas 11371833
N clientes 2525988
**********


In [ ]:
OrderMAtpd=df_orderMAttrib.select('SLA_factor_running').toPandas()

In [ ]:
print(OrderMAtpd)

In [ ]:
fig = plt.figure(figsize=(8, 5), dpi=100)
plt.hist(OrderMAtpd['SLA_factor_running'].dropna(),bins=30)
plt.show()

In [ ]:
df_orderMAttrib.filter((col('SLA_factor_running')>1) & (col('SLA_factor_running')<3)).select('NIF_CLIENTE').distinct().count()


In [ ]:
df_orderMAttrib.filter((col('SLA_factor_running')>3)).select('NIF_CLIENTE').distinct().count()

In [ ]:
df_orderMAttrib.filter((col('SLA_factor_running')<1)).select('NIF_CLIENTE').distinct().count()

In [ ]:
df_orderMAttrib.columns

In [ ]:
df_orderMAttrib.select('NUM_CLIENTE').distinct().count()

In [ ]:
classif=df_orderMAttrib.select('x_clasificacion').distinct()

In [ ]:
df_orderMAttrib.columns

In [ ]:
#df_SLA=df_orderMAttrib.select('NIF_CLIENTE').distinct()

#for dd in days_range:
    
#    df_orderSLA=(df_orderMAttrib.filter(col('Flag_open_order_insideSLA{}'.format(dd))==1)
#                 .groupBy('NIF_CLIENTE').pivot('x_clasificacion').agg(count('*').alias('NOrders_Opened{}_InsideSLA'.format(dd)),
#                                                                      sql_avg(col('running_days')).alias('Avg_runningDays{}_InsideSLA'.format(dd)),
#                                                                      sql_max(col('running_days')).alias('Max_runningDays{}_InsideSLA'.format(dd)),
#                                                                      sql_max(col('SLA_factor_running')).alias('Max_SLAfactor{}_InsideSLA'.format(dd))
#                                                                     ))    
    
#    df_SLA=df_SLA.join(df_orderSLA,['NIF_CLIENTE'],'outer')
    
#    df_orderISLA=(df_orderMAttrib.filter(col('Flag_open_order_outsideSLA{}'.format(dd))==1)
#                .groupBy('NIF_CLIENTE').pivot('x_clasificacion').agg(count('*').alias('NOrders_Opened{}_OutsideSLA'.format(dd)),
#                                                                     sql_avg(col('running_days')).alias('Avg_runningDays{}_OutsideSLA'.format(dd)),
#                                                                     sql_max(col('running_days')).alias('Max_runningDays{}_OutsideSLA'.format(dd)),
#                                                                    sql_max(col('SLA_factor_running')).alias('Max_SLAfactor{}_OutsideSLA'.format(dd))
 #                                                                   ))
    
#    df_SLA=df_SLA.join(df_orderISLA,['NIF_CLIENTE'],'outer')
    
#df_SLA=df_SLA.na.fill(0)

In [ ]:
df_SLA.columns

In [18]:
#df_SLA=df_orderMAttrib.select('NUM_CLIENTE').distinct()
df_orderSLA=(df_orderMAttrib.groupBy('NIF_CLIENTE').pivot('x_clasificacion')
             .agg(*([sql_sum(when(col('Flag_open_order_insideSLA{}'.format(dd))==1,1).otherwise(0)).alias('NOrders_Opened{}_InsideSLA'.format(dd)) for dd in days_range]+
                    [sql_sum(when(col('Flag_open_order_outsideSLA{}'.format(dd))==1,1).otherwise(0)).alias('NOrders_Opened{}_OutsideSLA'.format(dd)) for dd in days_range]+
                    [sql_avg(when(col('Flag_open_order_outsideSLA{}'.format(dd))==1,col('running_days')).otherwise(None)).alias('Avg_runningDays{}_OutsideSLA'.format(dd)) for dd in days_range]+
                    [sql_avg(when(col('Flag_open_order_insideSLA{}'.format(dd))==1,col('running_days')).otherwise(None)).alias('Avg_runningDays{}_InsideSLA'.format(dd)) for dd in days_range]+
                    [sql_max(when(col('Flag_open_order_outsideSLA{}'.format(dd))==1,col('running_days')).otherwise(0)).alias('Max_runningDays{}_OutsideSLA'.format(dd)) for dd in days_range]+
                    [sql_max(when(col('Flag_open_order_insideSLA{}'.format(dd))==1,col('running_days')).otherwise(0)).alias('Max_runningDays{}_InsideSLA'.format(dd)) for dd in days_range]+
                    [sql_max(when(col('Flag_open_order_outsideSLA{}'.format(dd))==1,col('SLA_factor_running')).otherwise(0)).alias('Max_SLA_factor_running{}_OutsideSLA'.format(dd)) for dd in days_range]+
                    [sql_max(when(col('Flag_open_order_insideSLA{}'.format(dd))==1,col('SLA_factor_running')).otherwise(0)).alias('Max_SLA_factor_running{}_InsideSLA'.format(dd)) for dd in days_range]+
                    [sql_avg(when(col('Flag_open_order_outsideSLA{}'.format(dd))==1,col('SLA_factor_running')).otherwise(None)).alias('Avg_SLA_factor_running{}_OutsideSLA'.format(dd)) for dd in days_range]+
                    [sql_avg(when(col('Flag_open_order_insideSLA{}'.format(dd))==1,col('SLA_factor_running')).otherwise(None)).alias('Avg_SLA_factor_running{}_InsideSLA'.format(dd)) for dd in days_range]
                    
                   ))
                      
            )

#df_orderSLA=df_orderSLA.na.fill(0)   



In [ ]:
dfMaxLabeled=(df_orderSLA.withColumn('Avg_'+str(day),greatest(*selColumns)).select(['nif_cliente','Avg_'+str(day)]))




In [ ]:
dfMaxLabeled.count()

In [ ]:
dfMaxLabeled.filter(col('Avg_30')<1).count()

In [ ]:
dfMaxLabeled.filter((col('Avg_30')>1) & (col('Avg_30')<3)).count()

In [ ]:
dfMaxLabeled.filter(col('Avg_30')>3).count()

In [ ]:
#df_SLA=df_orderMAttrib.select('NUM_CLIENTE').distinct()
df_orderSLA_prev=(df_orderMAttrib.groupBy('NIF_CLIENTE')
             .agg(*([sql_max(when(col('Flag_open_order_outsideSLA{}'.format(dd)) == 1,col('running_days')).otherwise(0)).alias('Max_runningDays{}_OutsideSLA'.format(dd)) for dd in days_range[:1]] +
                    [sql_max(when(col('Flag_open_order_insideSLA{}'.format(dd)) == 1,col('running_days')).otherwise(0)).alias('Max_runningDays{}_InsideSLA'.format(dd)) for dd in days_range[:1]] +
                    [sql_avg(when(col('Flag_open_order_outsideSLA{}'.format(dd))==1,col('SLA_factor_running')).otherwise(None)).alias('Avg_SLA_factor_running{}_OutsideSLA'.format(dd)) for dd in days_range[:1]]+
                    [sql_avg(when(col('Flag_open_order_insideSLA{}'.format(dd))==1,col('SLA_factor_running')).otherwise(None)).alias('Avg_SLA_factor_running{}_InsideSLA'.format(dd)) for dd in days_range[:1]]+
                    [sql_min(when(col('Flag_open_order_outsideSLA{}'.format(dd)) == 1,col('running_days')).otherwise(0)).alias('Min_runningDays{}_OutsideSLA'.format(dd)) for dd in days_range[:1]] +
                    [sql_min(when(col('Flag_open_order_insideSLA{}'.format(dd)) == 1,col('running_days')).otherwise(0)).alias('Min_runningDays{}_InsideSLA'.format(dd)) for dd in days_range[:1]]

                   ))
                      
            )

In [25]:
#df_SLA=df_orderMAttrib.select('NUM_CLIENTE').distinct()
df_orderSLA2=(df_orderMAttrib.groupBy('NIF_CLIENTE').pivot('x_clasificacion')
             .agg(*([sql_max(when(col('Flag_open_order_outsideSLA{}'.format(dd)) == 1,col('running_days')).otherwise(0)).alias('Max_runningDays{}_OutsideSLA'.format(dd)) for dd in days_range[:1]] +
                    [sql_max(when(col('Flag_open_order_insideSLA{}'.format(dd)) == 1,col('running_days')).otherwise(0)).alias('Max_runningDays{}_InsideSLA'.format(dd)) for dd in days_range[:1]] +
                    [sql_avg(when(col('Flag_open_order_outsideSLA{}'.format(dd))==1,col('SLA_factor_running')).otherwise(None)).alias('Avg_SLA_factor_running{}_OutsideSLA'.format(dd)) for dd in days_range[:1]]+
                    [sql_avg(when(col('Flag_open_order_insideSLA{}'.format(dd))==1,col('SLA_factor_running')).otherwise(None)).alias('Avg_SLA_factor_running{}_InsideSLA'.format(dd)) for dd in days_range[:1]]+
                    [sql_min(when(col('Flag_open_order_outsideSLA{}'.format(dd)) == 1,col('SLA_factor_running')).otherwise(0)).alias('Min_SLA_factor_running{}_OutsideSLA'.format(dd)) for dd in days_range[:1]] +
                    [sql_min(when(col('Flag_open_order_insideSLA{}'.format(dd)) == 1,col('SLA_factor_running')).otherwise(0)).alias('Min_SLA_factor_running{}_InsideSLA'.format(dd)) for dd in days_range[:1]]
                   ))
                      
            )

df_orderSLA2=df_orderSLA2.na.fill(-1)

In [ ]:
df_orderSLA_prev.select(['NIF_CLIENTE','Min_runningDays30_OutsideSLA','Max_runningDays30_OutsideSLA','Avg_SLA_factor_running30_OutsideSLA']).filter(col('Max_runningDays30_OutsideSLA')>0).show(50,False)




In [26]:
nif='09755682O'#'02710100O'#'09755682O'#'18867804T'
colSel=['x_clasificacion','SLA_factor_running','SLA_master','duration','running_days','days_since_completed','days_since_start','Flag_open_order_insideSLA30','Flag_open_order_outsideSLA30','Flag_open_order_outsideSLA60']
df_orderMAttrib.select(colSel).filter(col('NIF_CLIENTE')==nif).show()



+---------------+------------------+----------+--------+------------+--------------------+----------------+---------------------------+----------------------------+----------------------------+
|x_clasificacion|SLA_factor_running|SLA_master|duration|running_days|days_since_completed|days_since_start|Flag_open_order_insideSLA30|Flag_open_order_outsideSLA30|Flag_open_order_outsideSLA60|
+---------------+------------------+----------+--------+------------+--------------------+----------------+---------------------------+----------------------------+----------------------------+
|      ord_admin|               0.0|         1|       0|           0|                  68|              68|                          0|                           0|                           0|
|        aumento|              0.75|         4|       3|           3|                   4|               7|                          0|                           0|                           0|
|         cambio|             

In [27]:
day=30
print 'Range Day:',day
selColumns=[]
for column in df_orderSLA2.columns:
    if ('Avg_SLA_factor_running'+str(day) in column):# and ('Outside' in column):
        selColumns.append(column)


Range Day: 30


In [28]:
df_orderSLA2.select(selColumns).filter(col('NIF_CLIENTE')==nif).show()


+-------------------------------------------+------------------------------------------+------------------------------------------+-----------------------------------------+----------------------------------------------+---------------------------------------------+-----------------------------------------------+----------------------------------------------+-----------------------------------------------+----------------------------------------------+---------------------------------------------+--------------------------------------------+---------------------------------------------+--------------------------------------------+----------------------------------------------+---------------------------------------------+-------------------------------------------+------------------------------------------+--------------------------------------------+-------------------------------------------+----------------------------------------------+---------------------------------------------+

In [29]:
dfMaxLabeled2=(df_orderSLA2.withColumn('Max_30',greatest(*selColumns)).select(['nif_cliente','Max_30']))
dfMaxLabeled2.filter(col('NIF_CLIENTE')==nif).show()

+-----------+------+
|nif_cliente|Max_30|
+-----------+------+
|  09755682O|  1.75|
+-----------+------+



In [30]:
dfMinLabeled=(df_orderSLA2.withColumn('Min_30',least(*selColumns)).select(['nif_cliente','Min_30']))
dfMinLabeled.filter(col('NIF_CLIENTE')==nif).show()

+-----------+------+
|nif_cliente|Min_30|
+-----------+------+
|  09755682O|  -1.0|
+-----------+------+



In [ ]:
day=30
print 'Range Day:',day
selColumnsInside=[]
selColumnsOutside=[]
selColumns=[]
for column in df_orderSLA2.columns:
    if (('Inside' in column) and ('Avg' in column)):# and ('Outside' in column):
        selColumnsInside.append(column)
    
    if (('Outside' in column) and ('Avg' in column)):# and ('Outside' in column):
        selColumnsOutside.append(column)
        
    if ('Avg' in column):# and ('Outside' in column):
        selColumns.append(column)

In [ ]:
selColumnsOutside

In [32]:
remove_undefined_udf = udf(lambda milista: list([milista[ii] for ii in range(len(milista)) if milista[ii] !=None]), ArrayType(FloatType()))
mean_udf = udf(lambda milista: float(np.mean(milista)), FloatType())



In [33]:
# Todas las columnas
df_orderSLA3=df_orderSLA2.withColumn('arrayList', array([col_ for col_ in selColumns]))
df_orderSLA4=df_orderSLA3.withColumn('arrayListClean', remove_undefined_udf('arrayList'))


dfAvgLabeled=(df_orderSLA4.withColumn('Avg_30', mean_udf(col('arrayListClean'))))
dfAvgLabeled.filter(col('NIF_CLIENTE')==nif).show()


+-----------+------------------------------------+-----------------------------------+-------------------------------------------+------------------------------------------+-------------------------------------------+------------------------------------------+-----------------------------------+----------------------------------+------------------------------------------+-----------------------------------------+------------------------------------------+-----------------------------------------+---------------------------------------+--------------------------------------+----------------------------------------------+---------------------------------------------+----------------------------------------------+---------------------------------------------+----------------------------------------+---------------------------------------+-----------------------------------------------+----------------------------------------------+-----------------------------------------------+----------

In [ ]:
# Inside
df_orderSLA3=df_orderSLA2.withColumn('arrayList', array([col_ for col_ in selColumnsInside]))
df_orderSLA4=df_orderSLA3.withColumn('arrayListClean', remove_undefined_udf('arrayList'))


dfAvgLabeled2=(df_orderSLA4.withColumn('Avg_30', mean_udf(col('arrayListClean'))))
dfAvgLabeled2.filter(col('NIF_CLIENTE')==nif).show()

In [ ]:
# Outside
df_orderSLA3=df_orderSLA2.withColumn('arrayList', array([col_ for col_ in selColumnsOutside]))
df_orderSLA4=df_orderSLA3.withColumn('arrayListClean', remove_undefined_udf('arrayList'))


dfAvgLabeled2=(df_orderSLA4.withColumn('Avg_30', mean_udf(col('arrayListClean'))))
dfAvgLabeled2.filter(col('NIF_CLIENTE')==nif).show()

In [42]:
dfMaxLabeled2C=dfMaxLabeled2.filter(col('Max_30')>-1)#dropna(subset='Max_30')

print (dfMaxLabeled2C.filter(col('Max_30')<1).count(),
       dfMaxLabeled2C.filter((col('Max_30')>1) & (col('Max_30')<3)).count(),
       dfMaxLabeled2C.filter(col('Max_30')>3).count())



(42379, 9274, 29477)


In [41]:
dfMaxLabeled2C.filter(col('Max_30')<0).show()

+-----------+------+
|nif_cliente|Max_30|
+-----------+------+
|  33322399W|  -1.0|
|  BM4384667|  -1.0|
|  86088097U|  -1.0|
|  91154223O|  -1.0|
|  09441256H|  -1.0|
|  76387070A|  -1.0|
|  61957171G|  -1.0|
|  368291273|  -1.0|
|  41661364E|  -1.0|
|  48793716P|  -1.0|
|  57194284O|  -1.0|
|  71384428O|  -1.0|
|  97470524K|  -1.0|
|  83163847Z|  -1.0|
|  15004776I|  -1.0|
|  19925584Y|  -1.0|
|  28615985U|  -1.0|
|  Y2866822S|  -1.0|
|  J55384763|  -1.0|
|  99754069L|  -1.0|
+-----------+------+
only showing top 20 rows



In [39]:
dfMaxLabeled2C.filter((col('Max_30')>0) & (col('Max_30')<1)).count()

30638

In [ ]:
dfMaxLabeled2C.select(['NIF_CLIENTE','Max_30']).filter(col('Max_30')>3).show()

In [ ]:
dfMaxLabeled2C.select(['NIF_CLIENTE','Max_30']).filter(col('Max_30')>3).count()

In [ ]:
maxPd=dfMaxLabeled2C.select(['NIF_CLIENTE','Max_30']).toPandas()

In [ ]:
fig = plt.figure(figsize=(8, 5), dpi=100)
plt.hist(maxPd['Max_30'],bins=30)
plt.show()



In [ ]:

df_orderMAttrib.select(colSel).filter(col('NIF_CLIENTE')=='85309487B').show()


In [ ]:
dfMinLabeledC=dfMinLabeled.dropna(subset='Min_30')

print (dfMinLabeledC.filter(col('Min_30')<1).count(),
       dfMinLabeledC.filter((col('Min_30')>1) & (col('Min_30')<3)).count(),
       dfMinLabeledC.filter(col('Min_30')>3).count())

In [ ]:
dfAvgLabeled2.filter(col('NIF_CLIENTE')==nif).show()

In [ ]:
dfAvgLabeled2C=dfAvgLabeled2.dropna(subset='Avg_30')

print (dfAvgLabeled2C.filter(col('Avg_30')<1).count(),
       dfAvgLabeled2C.filter((col('Avg_30')>1) & (col('Avg_30')<3)).count(),
       dfAvgLabeled2C.filter(col('Avg_30')>3).count())

In [ ]:
dfAvgLabeled2C.select(['NIF_CLIENTE','Avg_30']).filter(col('Avg_30')>3).show()

In [ ]:
dfAvgLabeled2C.select(['Avg_30']).distinct().sort(col('Avg_30').desc()).show()

In [ ]:
df_orderSLA2.columns

In [ ]:
selColumns2=['Avg_SLA_factor_running30_OutsideSLA','Avg_SLA_factor_running30_InsideSLA']
dfMaxLabeled2=(df_orderSLA2.withColumn('Avg_'+str(day),greatest(*selColumns2)).select(['nif_cliente','Avg_'+str(day)]))


In [ ]:
dfMaxLabeled2.columns

In [ ]:
dfMaxLabeled2.filter(col('Avg_30')<1).count()

In [ ]:
dfMaxLabeled2.filter((col('Avg_30')>1) & (col('Avg_30')<3)).count()

In [ ]:
dfMaxLabeled2.filter(col('Avg_30')>3).count()

In [ ]:
del df_orderSLA

In [ ]:
df_orderMAttrib.columns

In [ ]:
df_orderSLA.filter((col('SLA_factor_running')>1) & (col('SLA_factor_running')<3)).select('NIF_CLIENTE').distinct().count()



In [ ]:
df_orderMAttrib.filter((col('SLA_factor_running')>3)).select('NIF_CLIENTE').distinct().count()

In [ ]:
df_orderMAttrib.filter((col('SLA_factor_running')<1)).select('NIF_CLIENTE').distinct().count()

In [ ]:
df_orderSLA.filter(col('NUM_CLIENTE')=='459530360').select(['ord_esp_Avg_runningDays30_InsideSLA','ord_esp_Avg_runningDays60_InsideSLA']).show()


In [ ]:
df_SLA.filter(col('NOrders_Opened30_OutsideSLA')!=col('NOrders_Opened120_OutsideSLA')).show()

In [ ]:
df_orderM.select(['NUM_CLIENTE','SLA_master','days_since_start','days_since_completed','duration','x_clasificacion']).filter(col('NUM_CLIENTE')=='459530360').show()



In [ ]:
df_orderMAttrib.select(['NUM_CLIENTE','SLA_master','running_days','SLA_factor_running','days_since_start','days_since_completed','duration','x_clasificacion']).filter(col('NUM_CLIENTE')=='459530360').show()


In [ ]:
df_orderM.select(['NUM_CLIENTE','SLA_master','days_since_start','days_since_completed','duration','x_clasificacion']).filter(col('NUM_CLIENTE')=='000509943').show()




In [ ]:
columnas.extend(['Flag_open_order_insideSLA30','Flag_open_order_insideSLA60','Flag_open_order_outsideSLA30','Flag_open_order_outsideSLA60'])



In [ ]:
df_orderMAttrib.select(columnas).filter(col('NUM_CLIENTE')=='459530360').show()

In [ ]:
df_orderMAttrib.select(columnas).filter(col('NUM_CLIENTE')=='000509943').show()

In [ ]:
df_orderMAttrib.select(columnas).filter(col('NUM_CLIENTE')=='459530360').show()

In [ ]:
df_orderMAttrib.select(['NUM_CLIENTE','NIF_CLIENTE']).filter(col('NUM_CLIENTE')=='459530360').show()

In [ ]:
dfSel1=SLAdf.filter(col('NIF_CLIENTE')=='76148857B')

In [ ]:
dfSLa=spark.read.parquet('/user/bgmerin1/SLA/year=2019/month=5/day=21')

In [ ]:
dfSLa.filter(col('NIF_CLIENTE')=='76148857B').show()

In [ ]:
len(dfSLa.columns)